In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [7]:
target = pd.read_csv('target2',header=None,index_col=0)

In [8]:
train = feather.read_dataframe('best_table\\train')
test = feather.read_dataframe('best_table\\test')

In [4]:
groupped = feather.read_dataframe('cats and percent spent\\groupped_table') 

train = pd.merge(train,groupped,how='left',on='card_id')
test = pd.merge(test,groupped,how='left',on='card_id')

In [5]:
train.head()

auth_purchase_amount_min  last_months_purchase_max  1to2_rel_mean  \
0                 -0.739395                      11.0       1.051026   
1                 -0.742400                       3.0       0.979470   
2                 -0.730138                       1.0       0.000000   
3                 -0.740897                       5.0       1.013385   
4                 -0.746156                      20.0       0.730606   

   agged_au_purchase_date_months_week_std  auth_purchase_amount_sum  \
0                                1.302837               -157.397016   
1                                1.274626               -208.883450   
2                                1.242866                -27.828424   
3                                1.325421                -49.491364   
4                                1.311943                -69.040465   

   month_lag_mean  all_month_pos__15  hist_month_lag_std  \
0       -4.000000               12.0            1.664101   
1       -6.000000                3.0            2.696799   
2       -6.666667                0.0            1.414214   
3       -2.500000                2.0                 NaN   
4       -1.500000               16.0            0.547723   

   auth_days_activity_max  all_month_fbte__9_prc  4to5_rel_mean  \
0                     243               0.042403       1.047633   
1                     391               0.073034       0.983653   
2                     413               0.136364       0.000000   
3                     155               0.000000       0.858085   
4                     109               0.000000       0.932046   

   all_months_distance_between_purchases_median  new_month_diff_mean  \
0                                         379.5             9.782609   
1                                         366.0            10.833333   
2                                         491.0             9.000000   
3                                         422.0            10.000000   
4                                         413.0             9.888889   

   id5_percent_of_purchases  mi_card_id_add_mode  \
0                  0.187682                350.0   
1                  0.051425              14645.0   
2                  0.000000               4105.0   
3                  0.045298             239115.0   
4                  0.029367                 73.0   

   new_merchant_category_id_nunique  1to2_rel_gen_mean  new_category_3_B_mean  \
0                              14.0           0.879227               0.000000   
1                               5.0           0.815372               1.000000   
2                               1.0           0.000000               0.000000   
3                               6.0           0.849149               0.857143   
4                              17.0           0.863797               0.944444   

   all_purchases_max_percent  hist_month_lag_mean  6to7_rel_gen_mean  \
0                   0.201413            -4.461538           0.851629   
1                   0.143258            -4.454545           0.947199   
2                   0.159091           -11.000000           0.842272   
3                   0.369048                  NaN           0.840349   
4                   0.272189            -0.400000           0.000000   

   hist_purchase_date_max  all_month_pos__6_prc  purchase_amount_sum_std  \
0            1.514385e+09              0.010601                10.015655   
1            1.514467e+09              0.042135                10.153878   
2            1.492801e+09              0.000000                 1.498500   
3                     NaN              0.000000                 7.140595   
4            1.519759e+09              0.000000                 8.316360   

   id6_percent_of_purchases  4to5_rel_gen_mean  authorized_flag_mean  \
0                  0.169598           0.168581              0.950000   
1                  0.138897           0.159641              0.968571   
2                  0.069731           0.

In [9]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + [x for x in features if re.search('[th]_cat1_[^s]',x)]

In [10]:
warnings.filterwarnings("ignore")
for col in [x for x in train2 if x!='card_id']:
    train[col] = train2[col]
    best_score = 3.65474
    features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
    categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] #+ [x for x in features if re.search('[th]_cat1_[^s]',x)]
    
    
    param = {
            'num_leaves': 111, #checked
             'num_threads': 4,
             'max_bin': 170, #checked 170
             'min_data_in_leaf': 100, #checked 100
             'objective':'regression',
             'max_depth': 9, #checked
             'tree_learner':'data',
             'learning_rate': 0.07,
             "boosting": "gbdt", #dart, gbdt
             "feature_fraction": 0.9, #checked
             "bagging_freq": 5, #5
             "bagging_fraction": 0.9, #checked
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.26, #checked 26
    #            'lambda_l2': 0.3, #checked
             "random_state": 77,
             "verbosity": -1,
             "device":"gpu",
             'gpu_device_id':0,
             'gpu_platform_id':1,
             'gpu_use_dp':True,
            }


    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))

        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats
                              )


        clf = lgb.train(param, trn_data,10000, valid_sets = [trn_data, val_data], verbose_eval=250, early_stopping_rounds = 200)

        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

#        predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
    errr = mean_squared_error(oof, target)**0.5
#    print("CV score: {:<8.5f}".format(errr))
    if errr > best_score:
        train.drop(col,axis=1,inplace=True)
        print('adds noise')
    else:
        best_score = errr
        print('a good column')
    
    print("CV score: {:<8.5f}\n0.{} - column\n".format(errr,col))
    
    #test 3.65598    n3 3.78153
    #0.90 frac: 3.64632  3.77624
    #0.75 frac: 3.64450  

NameError: name 'train2' is not defined

In [10]:
train.head()

auth_purchase_amount_min  last_months_purchase_max  1to2_rel_mean  \
0                 -0.739395                      11.0       1.051026   
1                 -0.742400                       3.0       0.979470   
2                 -0.730138                       1.0       0.000000   
3                 -0.740897                       5.0       1.013385   
4                 -0.746156                      20.0       0.730606   

   agged_au_purchase_date_months_week_std  auth_purchase_amount_sum  \
0                                1.302837               -157.397016   
1                                1.274626               -208.883450   
2                                1.242866                -27.828424   
3                                1.325421                -49.491364   
4                                1.311943                -69.040465   

   month_lag_mean  all_month_pos__15  hist_month_lag_std  \
0       -4.000000               12.0            1.664101   
1       -6.000000                3.0            2.696799   
2       -6.666667                0.0            1.414214   
3       -2.500000                2.0                 NaN   
4       -1.500000               16.0            0.547723   

   auth_days_activity_max  all_month_fbte__9_prc  4to5_rel_mean  \
0                     243               0.042403       1.047633   
1                     391               0.073034       0.983653   
2                     413               0.136364       0.000000   
3                     155               0.000000       0.858085   
4                     109               0.000000       0.932046   

   all_months_distance_between_purchases_median  new_month_diff_mean  \
0                                         379.5             9.782609   
1                                         366.0            10.833333   
2                                         491.0             9.000000   
3                                         422.0            10.000000   
4                                         413.0             9.888889   

   id5_percent_of_purchases  mi_card_id_add_mode  \
0                  0.187682                350.0   
1                  0.051425              14645.0   
2                  0.000000               4105.0   
3                  0.045298             239115.0   
4                  0.029367                 73.0   

   new_merchant_category_id_nunique  1to2_rel_gen_mean  new_category_3_B_mean  \
0                              14.0           0.879227               0.000000   
1                               5.0           0.815372               1.000000   
2                               1.0           0.000000               0.000000   
3                               6.0           0.849149               0.857143   
4                              17.0           0.863797               0.944444   

   all_purchases_max_percent  hist_month_lag_mean  6to7_rel_gen_mean  \
0                   0.201413            -4.461538           0.851629   
1                   0.143258            -4.454545           0.947199   
2                   0.159091           -11.000000           0.842272   
3                   0.369048                  NaN           0.840349   
4                   0.272189            -0.400000           0.000000   

   hist_purchase_date_max  all_month_pos__6_prc  purchase_amount_sum_std  \
0            1.514385e+09              0.010601                10.015655   
1            1.514467e+09              0.042135                10.153878   
2            1.492801e+09              0.000000                 1.498500   
3                     NaN              0.000000                 7.140595   
4            1.519759e+09              0.000000                 8.316360   

   id6_percent_of_purchases  4to5_rel_gen_mean  authorized_flag_mean  \
0                  0.169598           0.168581              0.950000   
1                  0.138897           0.159641              0.968571   
2                  0.069731           0.

In [ ]:
#3.65830  3.78166 no   mode 0.75 with_add
#3.65750  3.78065 with mode 0.75 with_add
#3.65545  3.78512 no   mode 0.9 with_add
#3.65498  3.78472 with mode 0.9 with_add

#3.65791  3.77967 0.75
#3.65474  3.77925 0.90

In [46]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [47]:
all_cols = pd.DataFrame(best_features.groupby(['feature'])['importance'].mean()).sort_values(by='importance')
all_cols

importance
feature                                            
new_category_3_B_mean                          35.2
hist_month_lag_std                             36.2
authorized_flag_mean                           37.0
all_month_fbte__9_prc                          37.0
all_purch_per_hour__13_prc                     39.6
hist_purchase_date_max                         40.0
3to4_rel_gen_mean                              41.4
all_month_pos__15                              42.0
purchase_amount_max_std                        42.4
all_month_fbte__4_prc                          42.4
auth_month_lag_std                             43.2
feature_1                                      43.2
all_month_fbte__6_prc                          43.4
mi_merchant_category_id_mean                   43.6
id3_percent_of_purchases                       43.8
new_days_activity_max                          43.8
purchase_amount_mean_mean                      44.6
6to7_rel_gen_mean                              44.6
category_combined_without_cat1_std             45.0
average_purchase_per_unique_day                46.6
auth_purchase_amount_sum                       47.2
city_id_purchase_amount_mean                   47.2
purchase_amount_min_std                        47.2
2to3_rel_mean                                  47.8
new_purchase_month_std                         47.8
installments_std_std                           48.0
installments_count_mean                        48.0
auth_purchase_amount_mean                      48.2
mi_card_id_add_mode                            48.4
purchase_amount_std_mean                       49.0
...                                             ...
all_purchases_mean_percent                     67.4
agged_all_purchase_date_year_mean              69.4
new_category_1_mean                            70.0
all_month_pos__14_prc                          70.4
agged_non_purchase_date_hour_std               70.8
average_purchase_per_day                       71.8
agged_all_merchant_category_id_modeplus        73.4
agged_all_category_1_sum                       73.6
category_combined_without_cat1_modeplus        75.0
all_month_fetb__1                              76.0
agged_non_city_id_modeplus                     79.8
agged_all_purchase_date_day_max                80.4
new_purchases_per_month_mean                   83.0
0to1_rel_gen_mean                              85.8
new_purchase_amount_std                        85.8
new_purchase_date_ptp                          86.2
new_purchase_date_min                          86.8
author_purchases_mean_percent                  86.8
agged_non_payment_per_month_sum                87.8
new_purchase_date_max                          88.4
purchase_amount_max_mean                       96.0
hist_month_diff_mean                          107.6
auth_purchase_date_max                        108.0
agged_au_purchase_date_day_max                112.6
last_payments_day                             118.8
agged_non_merchant_category_id_modeplus       123.2
auth_month_diff_mean                          129.6
new_purchase_month_mean                       149.0
new_purchase_amount_max                       158.6
category_combined_with_cat1_modeplus          168.2

[127 rows x 1 columns]

In [48]:
best_cols = all_cols[all_cols.importance > 40].index

In [49]:
del best_cols